<a href="https://colab.research.google.com/github/natexjake/Online-Courses-and-Tutorials-/blob/main/Udacity/Intro%20to%20TensorFlow%20for%20Deep%20Learning/Classifying_images_of_clothing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2018 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [ ]:
#@title MIT License
#
# Copyright (c) 2017 François Chollet
#
# Permission is hereby granted, free of charge, to any person obtaining a
# copy of this software and associated documentation files (the "Software"),
# to deal in the Software without restriction, including without limitation
# the rights to use, copy, modify, merge, publish, distribute, sublicense,
# and/or sell copies of the Software, and to permit persons to whom the
# Software is furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL
# THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING
# FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER
# DEALINGS IN THE SOFTWARE.

In [ ]:
!pip install -U tensorflow_datasets #installs or updates the tensorflow_datasets package.

In [ ]:
import tensorflow as tf #TensorFlow library is imported under the alias tf.

In [ ]:
# Import TensorFlow Datasets
import tensorflow_datasets as tfds #tensorflow_datasets library is imported under the alias tfds
tfds.disable_progress_bar() #progress bar display is disabled.

# Helper libraries
import math #Python's built-in math library
import numpy as np #numpy library is imported under the alias np,
import matplotlib.pyplot as plt #matplotlib.pyplot is imported under the alias plt

In [ ]:
import logging #logging module is imported
logger = tf.get_logger() #A logger object is created to control the logging level of TensorFlow
logger.setLevel(logging.ERROR) #set the logger to display only error messages.

In [ ]:
dataset, metadata = tfds.load('fashion_mnist', as_supervised=True, with_info=True) #loads the Fashion MNIST dataset using tensorflow_datasets. The as_supervised=True argument ensures that the dataset is returned in a supervised format (input, label), and with_info=True returns additional metadata about the dataset.
train_dataset, test_dataset = dataset['train'], dataset['test'] #The loaded dataset is split into train_dataset and test_dataset.

In [ ]:
class_names = metadata.features['label'].names #The class names of the Fashion MNIST dataset are extracted from the metadata
print("Class names: {}".format(class_names)) #prints the class names of the Fashion MNIST dataset

In [ ]:
#The number of training examples and test examples are retrieved from the metadata and printed.
num_train_examples = metadata.splits['train'].num_examples
num_test_examples = metadata.splits['test'].num_examples
print("Number of training examples: {}".format(num_train_examples))
print("Number of test examples:     {}".format(num_test_examples))

In [ ]:
#A function named normalize is defined. It takes images and labels as input
#and normalizes the images by dividing them by 255 (to scale the pixel values between 0 and 1).
#The normalized images and original labels are returned.
def normalize(images, labels):
  images = tf.cast(images, tf.float32)
  images /= 255
  return images, labels

# The map function applies the normalize function to each element in the train
# and test datasets
train_dataset =  train_dataset.map(normalize) #The normalize function is applied to each element (image, label) using the map() function.
test_dataset  =  test_dataset.map(normalize) #The normalize function is applied to each element (image, label) using the map() function.

# The first time you use the dataset, the images will be loaded from disk
# Caching will keep them in memory, making training faster
train_dataset =  train_dataset.cache()
test_dataset  =  test_dataset.cache()

In [ ]:
# Take a single image, and remove the color dimension by reshaping
# A single image and its corresponding label are extracted from the test_dataset.
# The image is converted to a NumPy array and reshaped to (28, 28) dimensions.
for image, label in test_dataset.take(1):
  break
image = image.numpy().reshape((28,28))

# Plot the image - voila a piece of fashion clothing
#The image is displayed using matplotlib.pyplot.imshow().
#It visualizes the image as grayscale using the 'binary' color map.
#A color bar, grid, and axis ticks are added to the plot, and it is displayed using plt.show().
plt.figure()
plt.imshow(image, cmap=plt.cm.binary)
plt.colorbar()
plt.grid(False)
plt.show()

In [ ]:
#A grid of 25 images is displayed using matplotlib.pyplot.subplot().
#Each image is retrieved from the train_dataset and reshaped to (28, 28) dimensions.
#The images are shown without axis ticks and gridlines, and the class label is displayed as the xlabel.

plt.figure(figsize=(10,10))
for i, (image, label) in enumerate(train_dataset.take(25)):
    image = image.numpy().reshape((28,28))
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(image, cmap=plt.cm.binary)
    plt.xlabel(class_names[label])
plt.show()

In [ ]:
#A sequential model is created using tf.keras.Sequential().
#It consists of three layers: a flattening layer to convert the 2D image into a 1D array,
#a dense layer with 128 units and ReLU activation function,
#and a dense layer with 10 units (corresponding to the number of classes) and softmax activation function.

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28, 1)),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])

In [ ]:
#The model is compiled with the Adam optimizer, sparse categorical cross-entropy loss function,
#and accuracy as the evaluation metric.
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

In [ ]:
#The training dataset is cached, repeated, shuffled, and batched with the specified BATCH_SIZE.
#The test dataset is cached and batched.

BATCH_SIZE = 32
train_dataset = train_dataset.cache().repeat().shuffle(num_train_examples).batch(BATCH_SIZE)
test_dataset = test_dataset.cache().batch(BATCH_SIZE)

In [ ]:
#The model is trained using the fit() function on the train_dataset.
#Training is performed for 5 epochs, and the number of steps per epoch is calculated based on the number of training examples and the batch size.
model.fit(train_dataset, epochs=5, steps_per_epoch=math.ceil(num_train_examples/BATCH_SIZE))

In [ ]:
#The model is evaluated on the test dataset using the evaluate() function.
#The test loss and accuracy are calculated and printed.
test_loss, test_accuracy = model.evaluate(test_dataset, steps=math.ceil(num_test_examples/32))
print('Accuracy on test dataset:', test_accuracy)

In [ ]:
#A batch of test images and their labels are extracted from the test_dataset.
#The images and labels are converted to NumPy arrays.
#Predictions for the test images are obtained using the trained model.

for test_images, test_labels in test_dataset.take(1):
  test_images = test_images.numpy()
  test_labels = test_labels.numpy()
  predictions = model.predict(test_images)

In [ ]:
#The shape of the predictions array is printed
predictions.shape


In [ ]:
predictions[0]  #the first prediction is accessed

In [ ]:
#the index of the maximum predicted value is obtained,

In [ ]:
test_labels[0] #the corresponding true label is retrieved.

In [ ]:
# A function named plot_image() is defined. It takes an index i, predictions array, true labels,
# and images as input. It plots and visualizes the image with its predicted and true labels.
# If the predicted label matches the true label, the label text is shown in blue; otherwise, it is shown in red.


def plot_image(i, predictions_array, true_labels, images):
  predictions_array, true_label, img = predictions_array[i], true_labels[i], images[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])

  plt.imshow(img[...,0], cmap=plt.cm.binary)

  predicted_label = np.argmax(predictions_array)
  if predicted_label == true_label:
    color = 'blue'
  else:
    color = 'red'

  plt.xlabel("{} {:2.0f}% ({})".format(class_names[predicted_label],
                                100*np.max(predictions_array),
                                class_names[true_label]),
                                color=color)

# A function named plot_value_array() is defined. It takes an index i, predictions array,
# and true label as input. It plots and visualizes a bar chart showing the predicted probabilities
# for each class. The predicted label is highlighted in red, and the true label is highlighted in blue.
def plot_value_array(i, predictions_array, true_label):
  predictions_array, true_label = predictions_array[i], true_label[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])
  thisplot = plt.bar(range(10), predictions_array, color="#777777")
  plt.ylim([0, 1])
  predicted_label = np.argmax(predictions_array)

  thisplot[predicted_label].set_color('red')
  thisplot[true_label].set_color('blue')

In [ ]:
# An image and its corresponding value array are plotted
# and displayed using the plot_image() and plot_value_array() functions.
i = 0
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plot_image(i, predictions, test_labels, test_images)
plt.subplot(1,2,2)
plot_value_array(i, predictions, test_labels)

In [ ]:
# Another image and its corresponding value array are plotted and displayed.
i = 12
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plot_image(i, predictions, test_labels, test_images)
plt.subplot(1,2,2)
plot_value_array(i, predictions, test_labels)

In [ ]:
# Plot the first X test images, their predicted label, and the true label
# Color correct predictions in blue, incorrect predictions in red
# A grid of multiple images and their corresponding value arrays is plotted and displayed.
num_rows = 5
num_cols = 3
num_images = num_rows*num_cols
plt.figure(figsize=(2*2*num_cols, 2*num_rows))
for i in range(num_images):
  plt.subplot(num_rows, 2*num_cols, 2*i+1)
  plot_image(i, predictions, test_labels, test_images)
  plt.subplot(num_rows, 2*num_cols, 2*i+2)
  plot_value_array(i, predictions, test_labels)


In [ ]:
# Grab an image from the test dataset
img = test_images[0]

print(img.shape)

In [ ]:
# Add the image to a batch where it's the only member.
img = np.array([img])

print(img.shape)

In [ ]:
predictions_single = model.predict(img)

print(predictions_single)

In [ ]:
# The value array for the single image is plotted
plot_value_array(0, predictions_single, test_labels)
_ = plt.xticks(range(10), class_names, rotation=45)

In [ ]:
# The predicted label for the single image is obtained by finding the index with the maximum predicted value.
np.argmax(predictions_single[0])

# Exercises

Experiment with different models and see how the accuracy results differ. In particular change the following parameters:
*   Set training epochs set to 1
*   Number of neurons in the Dense layer following the Flatten one. For example, go really low (e.g. 10) in ranges up to 512 and see how accuracy changes
*   Add additional Dense layers between the Flatten and the final `Dense(10)`, experiment with different units in these layers
*   Don't normalize the pixel values, and see the effect that has


Remember to enable GPU to make everything run faster (Runtime -> Change runtime type -> Hardware accelerator -> GPU).
Also, if you run into trouble, simply reset the entire environment and start from the beginning:
*   Edit -> Clear all outputs
*   Runtime -> Reset all runtimes